# Word Embeddings : le modèle Word2Vec

## Imports

In [1]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [2]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [3]:
infile = f"../data/sents.txt"
sentences = MySentences(infile)

### Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [4]:
bigram_phrases = Phrases(sentences)

L'object `phrases` peut être vu comme un large dictionnaire d'expressions multi-mots associées à un score, le *PMI-like scoring*. Ce dictionnaire est construit par un apprentissage sur base d'exemples.
Voir les références ci-dessous :
- https://arxiv.org/abs/1310.4546
- https://en.wikipedia.org/wiki/Pointwise_mutual_information

In [5]:
type(bigram_phrases.vocab)

dict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [6]:
len(bigram_phrases.vocab.keys())

1125226

Prenons une clé au hasard :

In [7]:
key_= list(bigram_phrases.vocab.keys())[200]
print(key_)

editions_:


Le dictionnaire indique le score de cette coocurrence :

In [8]:
bigram_phrases.vocab[key_]

131

Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [9]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui convertit certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [10]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [11]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [12]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [13]:
print(corpus[:100])

[['a', "*'", 'cw', "'-", 'tm', 'p', '^', '*\'*"', "'", ';', '"*#', "'*", '!'], ["'", '.'], ["'-#)#?"], ["'", '*>.'], ["*/'", "*'", "'", ':', "'", "'*'''##'.", 'h', ';', '5j', ";#'*", 'y', 'i', "('", "'", ',', 'v', "'", '*:', ':', "'", "'-", "'", '#', "'", '*\'#"*\'', '\'"\'\'', '#-', '#', '*', '^^#', '1', '"', ';"', "*'", '>>', 's', '.', 'iiifw', 'v', '.', ',', 'tf', "'^", '-', '1', "--'", '\'--"', '*#', "'*", '#.'], ['c', '-"', 'r', '^>>', 'ywwiis', '".'], ['#', '-', "'", 'rfytjfrtfiyprfc', '!', 'kur7', '*', 'vn', '?,', '16_pages', '-', '30', 'c', 'mes', '}?'], ['#', '\\', '<', 'en', 'belgique', 'et', 'allemagne', 'occupee', ')', 'ja', 'l', 'l', "'", 'etranger', 't', '3s', 'centimes', '*/', 'm', 'juin', '1929', ';', 'edit', '.'], ['****', '*_*', '*_*', 'a', '14_b', '.'], ['1', '*', '4_editions', ':', "'", 'ii', 'mit', '"', '*_edit', ',', 'de', 'notl', '.'], [';;#', 'abonnements', '!'], ['agglomeration_bruxelloise', '.<<', '.._..', '(', 'par', 'mon', '.'], ['7', '.', '50', "[;#/>',", '

## Entrainement d'un modèle Word2Vec sur ce corpus

In [14]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)

CPU times: total: 2.67 s
Wall time: 7.09 s


#### Remarque

Vous voyez ici que l'entrainement du modèle est parallélisé (sur 4 workers).

Lors qu'on parallélise l'entrainement du modèle, 4 modèles "séparés" sont entrainés sur environ un quart des phrases.

Ensuite, les résultats sont agrégés pour ne plus faire qu'un seul modèle.

On ne peut prédire quel worker aura quelle phrase, car il y a des aléas lors de la parallélisation (p. ex. un worker qui serait plus lent, etc.).

Du coup, les valeurs peuvent varier légèrement d'un entrainement à l'autre.

Mais, globalement, les résultats restent cohérents.

### Sauver le modèle dans un fichier

In [15]:
outfile = f"../data/newspapers.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [16]:
model = Word2Vec.load("../data/newspapers.model")

### Imprimer le vecteur d'un terme

In [18]:
model.wv["gouvernementaux"]

array([ 0.47520477,  0.16519159,  0.8124408 ,  0.5263848 ,  0.09036949,
       -0.24066353, -0.04775346,  0.09332875, -0.33433056,  0.5205435 ,
        1.0853317 , -0.77288663,  0.19262521, -0.16077514, -0.6896437 ,
        0.13996719, -0.16189687, -0.6576276 , -0.20127527,  0.33788684,
        0.64450294,  1.1386933 ,  0.95661545, -0.07663769, -0.2747037 ,
       -0.16424029, -0.3547174 , -0.5796415 , -0.29220584, -0.6169277 ,
       -0.7139925 , -0.47298   ], dtype=float32)

### Calculer la similarité entre deux termes

In [19]:
model.wv.similarity("ministre", "roi")

0.6546433

### Chercher les mots les plus proches d'un terme donné

In [20]:
model.wv.most_similar("ministre", topn=10)

[('ministere', 0.8374603390693665),
 ('premier_ministre', 0.8341019749641418),
 ('president_du_conseil', 0.8340783715248108),
 ('departement', 0.8297356367111206),
 ('ministre_des_finances', 0.8198961615562439),
 ('chancelier', 0.8150020241737366),
 ('president', 0.7994568943977356),
 ('delegue', 0.7827120423316956),
 ('rapporteur', 0.778166651725769),
 ('ministre_des_affaires', 0.7569368481636047)]

### Faire des recherches complexes à travers l'espace vectoriel

In [21]:
print(model.wv.most_similar(positive=['paris', 'londres'], negative=['belgique']))

[('berlin', 0.8170552849769592), ('tokio', 0.8021875619888306), ('budapest', 0.7922147512435913), ('vienne', 0.78741854429245), ('belgrade', 0.7727641463279724), ('bucarest', 0.7681686282157898), ('rome', 0.7647550106048584), ('prague', 0.7643837928771973), ('mercredi', 0.7556272745132446), ('washington', 0.7388765811920166)]
